# Playground notebook

## Import Packages

In [1]:
import tensorflow as tf
import wandb
from tensorflow import keras
import os
import numpy as np
from official.nlp import optimization
import tensorflow_hub as hub
import tensorflow_text as text
from asgard.utils.data_loader import load_datasets
from asgard.callbacks.callbacks import EarlyStoppingHammingScore
from asgard.metrics.metrics import HammingScoreMetric

2023-07-06 19:36:01.936049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 19:36:02.585767: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 19:36:05.246351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.1/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda-12.1/lib64:
2023-

## Load Dataset and Model

In [2]:
def get_class_weight(train_set, class_weight_kind="balanced"):
  if (class_weight_kind is None) or (class_weight_kind == "None"):
    class_weights = None

  elif class_weight_kind == "balanced":
    class_weights = compute_class_weights(train_set)

  elif class_weight_kind == "two-to-one":
    class_weights = np.zeros((16, 2))
    class_weights[:, 0] = 1.0
    class_weights[:, 1] = 2.0

  return class_weights


def get_weighted_loss(weights):
  def weighted_loss(y_train, y_pred):
    return keras.backend.mean(
      (weights[:, 0] ** (1 - y_train))
      * (weights[:, 1] ** y_train)
      * keras.backend.binary_crossentropy(y_train, y_pred),
      axis=-1,
    )

  return weighted_loss

In [3]:
# Define number of epochs
epochs = 4
steps_per_epoch = 24813 # tf.data.experimental.cardinality(train_set).numpy()

# Define optimizer
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.05 * num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

tfhub_encoder_handler = "https://tfhub.dev/jeongukjae/distilbert_en_uncased_L-6_H-768_A-12/1"
tfhub_preprocess_handler = "https://tfhub.dev/jeongukjae/distilbert_en_uncased_preprocess/2"

[2023-07-06 19:36:09] absl - INFO: using Adamw optimizer
[2023-07-06 19:36:09] absl - INFO: AdamWeightDecay gradient_clip_norm=1.000000


In [4]:
train_set, valid_set, test_set = load_datasets("../storage/datasets/tf_raw")

2023-07-06 19:36:11.858658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 19:36:12.368991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 19:36:12.370830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 19:36:12.374879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

In [5]:
target_folder = '../api/model/model-avid-sweep-1:v2'

weights = np.zeros((16, 2))
weights[:, 0] = 1.
weights[:, 1] = 2.

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=init_lr,
    decay_steps=num_train_steps,
    end_learning_rate=0.0,
    power=1.0)

model = tf.keras.models.load_model(target_folder,
                                   custom_objects={"weighted_loss": get_weighted_loss(weights),
                                                   "AdamWeightDecay": optimization.create_optimizer(init_lr=init_lr,
                                                                                                    num_train_steps=num_train_steps,
                                                                                                    num_warmup_steps=num_warmup_steps,
                                                                                                    optimizer_type='adamw'),
                                                   "WarmUp": optimization.WarmUp(initial_learning_rate=init_lr,
                                                                                 decay_schedule_fn=lr_schedule,
                                                                                 warmup_steps=num_warmup_steps),
                                                   "HammingScoreMetric": HammingScoreMetric()}
                                   )

[2023-07-06 19:36:14] absl - INFO: using Adamw optimizer
[2023-07-06 19:36:14] absl - INFO: AdamWeightDecay gradient_clip_norm=1.000000
[2023-07-06 19:36:36] absl - INFO: AdamWeightDecay gradient_clip_norm=1.000000


## Explainable AI

In [6]:
sample = "Law enforcement effects on marine life preservation in the South Pacific"

In [25]:
len(set(sample.split(" ")))

11

In [7]:
import eli5
import pandas as pd
from eli5.lime import TextExplainer
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# vec = CountVectorizer(ngram_range=(1, 2), token_pattern='(?u)\\b\\w+\\b')
# vec.fit([sample])

In [9]:
# vec.get_feature_names_out()

In [29]:

te = TextExplainer(random_state=42, n_samples=500)
te.fit(sample, model.predict)

16/16 [==============================] - 2s 156ms/step


/home/ubuntu/.pyenv/versions/3.9.13/envs/asgard/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


TextExplainer(char_based=False,
              clf=SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',
                                random_state=RandomState(MT19937) at 0x7F73C067D540),
              n_samples=500, random_state=42,
              sampler=MaskingTextSamplers(random_state=RandomState(MT19937) at 0x7F73C067D540,
                                          sampler_params=[{'bow': False},
                                                          {'bow': True}],
                                          token_pattern='(?u)\\b\\w+\\b',
                                          weights=array([0.7, 0.3])),
              token_pattern='(?u)\\b\\w+\\b',
              vec=CountVectorizer(ngram_range=(1, 2),
                                  token_pattern='(?u)\\b\\w+\\b'))

In [11]:
explanation = te.explain_weights(target_names=[f"SDG {i+1}" for i in range(16)],
                                 feature_names=te.vec_.get_feature_names_out(), top=10)

In [27]:
te.show_weights(target_names=[f"SDG {i+1}" for i in range(16)], feature_names=te.vec_.get_feature_names_out())

In [22]:
pd.read_html(
    eli5.show_weights(te.clf_, 
                      target_names=[f"SDG {i+1}" for i in range(16)], 
                      feature_names=te.vec_.get_feature_names_out())
    .data
    )[14]

,Weight?,Feature
0,1.338,marine
1,0.937,law enforcement
2,0.912,life preservation
3,0.744,in the
4,0.700,enforcement effects
5,0.603,marine life
6,0.494,south pacific
7,0.311,effects on
8,0.017,on marine
9,-0.104,preservation


In [8]:
# class AsgardNormalized(tf.keras.Model):
#     def __init__(self, model, **kwargs):
#         super().__init__(**kwargs)
#         self.model = model
        
#     def call(self, inputs):
#         return self.model(inputs)
        
#     def predict(self, inputs):
#         if len(inputs) == 1:
#             inputs = [inputs]
#             inputs = [tf.constant([input], dtype=tf.string) for input in inputs]
#         # else: 
#             # inputs = [tf.constant(input, dtype=tf.string) for input in inputs]

#         probabilities = self.model(inputs)
#         norm = tf.reduce_sum(probabilities)
#         normalized_probabilities = tf.divide(probabilities, norm)
#         return normalized_probabilities
# normalized_model = AsgardNormalized(model)